In [1]:
#import dependencies
import backtrader as bt
import alpaca_backtrader_api as api
import alpaca_trade_api as tradeapi
import sys
sys.path.insert(0, '../')
import pandas as pd
import talib as ta
import numpy as np
from config import key_id, secret_key
from datetime import datetime, timedelta



In [2]:
#import pickls 5 symbols
df1=pd.read_pickle('GOOG.pickle')
df2=pd.read_pickle('AMD.pickle')
df3=pd.read_pickle('AAP.pickle')

In [3]:
#form into dataframe list
dflist=[df1,df2,df3]

In [32]:
#write strategy with i in it for iteration later
class volumestrat321(bt.Strategy):
    """
        this strategy is based around the idea of generating a lot of potential buys and make tiny profits on each trade
        we recommend to have at least 60 parallel trades at any time to cover non avoidable losses.
        Recommended is to only sell based on ROI for this strategy
    """ 
#populate a dataframe with indicators
    
    
    
#------------------------------------strategy starts here--------------------------------------------------------------    
    def __init__(self):
        #populate exponential moving averages
        self.emahigh=bt.indicators.EMA(self.data.high, period=4)
        self.emaclose=bt.indicators.EMA(self.data.close, period=4)
        self.emalow=bt.indicators.EMA(self.data.low, period=4)
        
        self.rsi=bt.talib.RSI(self.data.close, timeperiod=14 )
        self.mfi=bt.talib.MFI(self.data.high,self.data.low,self.data.close,self.data.volume, timeperiod=14 )
        self.ultosc=bt.talib.ULTOSC(self.data.high,self.data.low,self.data.close)
        
        self.minusdi=bt.talib.MINUS_DI(self.data.high,self.data.low,self.data.close)
        self.plusdi=bt.talib.PLUS_DI(self.data.high,self.data.low,self.data.close)
     
        
        #creat stochastic momentum indicators 
        #1 for down, -1 for up
        #self.fastcross=bt.indicators.CrossOver(self.fastk, self.fastd)
        
        
        #create Average Directional Movement Index
        self.adx=bt.talib.ADX(self.data.high, self.data.low, self.data.close, timeperiod=14)
        
        
# if open score is less than ema_low, adx is greater than 30, fastk fastd less than 30, then buy
#uses indicators to determine buy and sell

    def next(self):
        if not self.position: #not in the market
            if self.adx > 22 and \
            self.mfi < 25 and \
            self.ultosc < 25 and \
            self.rsi < 32  and \
            self.minusdi > self.plusdi and \
            self.minusdi > 25: \
            #self.minusdi > self.plusdi:
                self.buy(size=20)
                
                
        else:
            if self.mfi > 64  and \
            self.adx > 22 and \
            self.plusdi > 25 and \
            self.minusdi < self.plusdi and \
            self.rsi > 73:
                self.sell(size=20)

In [33]:
# Run over everything
SQN_dict_list=[] 
tradeanalysis_dict_list=[]
ret_dict_list=[]
trans_dict_list=[]
pnl_list=[]

In [34]:
#create backtrader
cerebro = bt.Cerebro()

#cerebro.addstrategy(SmaCross)
#cerebro.addstrategy(Scalp)

#Add slippage
cerebro.broker = bt.brokers.BackBroker(slip_perc=0.005) 

#set up cerebro

cerebro.broker.setcash(100000)

cerebro.broker.setcommission(commission=0.0)
cerebro.addsizer(bt.sizers.PercentSizer, percents=50)

# Add the analyzers we are interested in
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name="ta")
cerebro.addanalyzer(bt.analyzers.SQN, _name="sqn")
cerebro.addanalyzer(bt.analyzers.Returns, _name="ret")
cerebro.addanalyzer(bt.analyzers.PeriodStats, _name="pstat")
cerebro.addanalyzer(bt.analyzers.Transactions, _name="trans")


# Add the observers we are interested in 
#this adds more things to the plot on top of the stocks chart
cerebro.addobserver(bt.observers.Broker)
cerebro.addobserver(bt.observers.BuySell)




cerebro.addstrategy(volumestrat321)
datacere = bt.feeds.PandasData(dataname=dflist[0])
cerebro.adddata(datacere)
#    print(day_list[i]+' added')
test=cerebro.run()





SQN_dict=test[0].analyzers.sqn.get_analysis() 
SQN_dict_list.append(SQN_dict)

tradeanalysis_dict=test[0].analyzers.ta.get_analysis()
tradeanalysis_dict_list.append(tradeanalysis_dict)

ret_dict=test[0].analyzers.ret.get_analysis()
ret_dict_list.append(ret_dict)

trans_dict=test[0].analyzers.trans.get_analysis()
trans_dict_list.append(trans_dict_list)

In [35]:
SQN_dict

AutoOrderedDict([('sqn', 12.619074898502012), ('trades', 1803)])

In [36]:
tradeanalysis_dict

AutoOrderedDict([('total',
                  AutoOrderedDict([('total', 1804),
                                   ('open', 1),
                                   ('closed', 1803)])),
                 ('streak',
                  AutoOrderedDict([('won',
                                    AutoOrderedDict([('current', 15),
                                                     ('longest', 20)])),
                                   ('lost',
                                    AutoOrderedDict([('current', 0),
                                                     ('longest', 6)]))])),
                 ('pnl',
                  AutoOrderedDict([('gross',
                                    AutoOrderedDict([('total',
                                                      105820.27999999987),
                                                     ('average',
                                                      58.691225734886224)])),
                                   ('net',
                     